In [ ]:
cd /content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data

/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data


In [ ]:
import pandas as pd

train_df = pd.read_csv('output_train.csv')
valid_df = pd.read_csv('output_val.csv')

In [ ]:
train_df.head()

,file,frame,label,nose_x,nose_y,nose_score,left_eye_x,left_eye_y,left_eye_score,right_eye_x,...,left_knee_score,right_knee_x,right_knee_y,right_knee_score,left_ankle_x,left_ankle_y,left_ankle_score,right_ankle_x,right_ankle_y,right_ankle_score
0,02077_H_A_N_C1.mp4,0,0,433.201904,137.339722,0.123687,433.536316,133.542877,0.128915,435.075867,...,0.992070,471.505096,269.176941,0.983481,459.491669,322.209106,0.983306,476.900970,311.007446,0.971570
1,02077_H_A_N_C1.mp4,1,0,432.706665,139.028076,0.119133,432.644653,135.262482,0.107487,434.446259,...,0.990311,471.737030,269.089661,0.984615,457.475311,320.276947,0.979955,477.392151,310.526978,0.972040
2,02077_H_A_N_C1.mp4,2,0,432.811005,140.320831,0.082605,432.059906,136.109604,0.069186,433.391357,...,0.989685,470.083954,268.875702,0.983896,456.548462,320.828400,0.976981,473.975281,310.356018,0.968025
3,02077_H_A_N_C1.mp4,3,0,431.418213,141.639893,0.084149,430.199036,137.462433,0.066609,431.576538,...,0.990412,470.245453,267.599304,0.986282,456.372253,320.752991,0.978816,474.933746,308.751160,0.972351
4,02077_H_A_N_C1.mp4,4,0,425.485809,142.749344,0.093535,424.375854,138.612152,0.070890,425.632202,...,0.989743,471.258270,266.028259,0.987877,456.317413,321.318939,0.978681,476.850220,307.429901,0.975985


In [ ]:
valid_df.head()

,file,frame,label,nose_x,nose_y,nose_score,left_eye_x,left_eye_y,left_eye_score,right_eye_x,...,left_knee_score,right_knee_x,right_knee_y,right_knee_score,left_ankle_x,left_ankle_y,left_ankle_score,right_ankle_x,right_ankle_y,right_ankle_score
0,02366_H_A_SY_C7.mp4,0,0,713.038818,227.906616,0.888511,717.906677,223.113281,0.924195,712.661621,...,0.977322,726.436951,385.376892,0.880025,731.935120,442.980225,0.940621,733.062866,437.023102,0.809121
1,02366_H_A_SY_C7.mp4,1,0,713.864014,227.872955,0.891030,718.779541,223.278336,0.923748,713.481201,...,0.977421,723.235352,385.598083,0.885203,732.559082,442.147339,0.941275,729.728516,437.006561,0.816043
2,02366_H_A_SY_C7.mp4,2,0,713.848267,228.159546,0.878212,718.976013,223.683899,0.919033,714.304138,...,0.973288,724.934143,384.681549,0.852213,729.944092,440.952271,0.927905,731.427673,435.665894,0.765373
3,02366_H_A_SY_C7.mp4,3,0,714.660950,228.523041,0.877201,720.220764,224.102509,0.916390,714.938904,...,0.973639,726.083923,385.064911,0.850723,730.536621,440.957397,0.929609,731.798218,435.709595,0.765879
4,02366_H_A_SY_C7.mp4,4,0,714.568359,229.067261,0.866399,720.067688,224.768997,0.911367,715.088623,...,0.971766,726.226807,386.860168,0.835380,729.281128,442.915710,0.924214,731.149902,437.982208,0.744962


In [ ]:
# Mean Joint Velocity (프레임 간 관절 위치 변화량 평균)
import numpy as np

joint_names = [
    'nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear',
    'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow',
    'left_wrist', 'right_wrist',
    'left_hip', 'right_hip', 'left_knee', 'right_knee',
    'left_ankle', 'right_ankle'
]

joint_cols = [(f'{j}_x', f'{j}_y') for j in joint_names]

def compute_mean_joint_velocity(df):
    result = np.zeros(len(df))

    for file in df['file'].unique():
        sub = df[df['file'] == file]

        vel = []
        for x, y in joint_cols:
            dx = sub[x].diff()
            dy = sub[y].diff()
            vel.append(np.sqrt(dx**2 + dy**2))

        mean_vel = np.mean(vel, axis=0)
        mean_vel = np.nan_to_num(mean_vel, nan=0.0)

        result[sub.index] = mean_vel

    return result

In [ ]:
# Mean Angular Velocity (관절 각속도)
angle_triplets = [
    ('left_shoulder', 'left_elbow', 'left_wrist'),
    ('right_shoulder', 'right_elbow', 'right_wrist'),
    ('left_hip', 'left_knee', 'left_ankle'),
    ('right_hip', 'right_knee', 'right_ankle'),
    ('left_shoulder', 'left_hip', 'left_knee'),
    ('right_shoulder', 'right_hip', 'right_knee')
]

def compute_angle(p1, p2, p3):
    v1 = p1 - p2
    v2 = p3 - p2
    cos = np.sum(v1 * v2, axis=1) / (
        np.linalg.norm(v1, axis=1) * np.linalg.norm(v2, axis=1) + 1e-6
    )
    return np.arccos(np.clip(cos, -1.0, 1.0))

def compute_mean_angular_velocity(df):
    result = np.zeros(len(df))

    for file in df['file'].unique():
        sub = df[df['file'] == file]

        ang_vels = []
        for a, b, c in angle_triplets:
            p1 = sub[[f'{a}_x', f'{a}_y']].values
            p2 = sub[[f'{b}_x', f'{b}_y']].values
            p3 = sub[[f'{c}_x', f'{c}_y']].values

            angle = compute_angle(p1, p2, p3)
            d_angle = np.abs(np.diff(angle, prepend=angle[0]))
            ang_vels.append(d_angle)

        mean_ang = np.mean(ang_vels, axis=0)
        mean_ang = np.nan_to_num(mean_ang, nan=0.0)

        result[sub.index] = mean_ang

    return result

In [ ]:
# Com Velocity (무게 중심 변화량)
def compute_com_velocity(df):
    result = np.zeros(len(df))

    for file in df['file'].unique():
        sub = df[df['file'] == file]

        com_x = (sub['left_hip_x'] + sub['right_hip_x']) / 2
        com_y = (sub['left_hip_y'] + sub['right_hip_y']) / 2

        dx = com_x.diff()
        dy = com_y.diff()

        vel = np.sqrt(dx**2 + dy**2)
        vel = np.nan_to_num(vel, nan=0.0)

        result[sub.index] = vel

    return result

In [ ]:
train_df['mean_joint_velocity'] = compute_mean_joint_velocity(train_df)
train_df['mean_angular_velocity'] = compute_mean_angular_velocity(train_df)
train_df['com_velocity'] = compute_com_velocity(train_df)

train_df[['mean_joint_velocity', 'mean_angular_velocity', 'com_velocity']].describe()

,mean_joint_velocity,mean_angular_velocity,com_velocity
count,415000.000000,415000.000000,415000.000000
mean,8.967456,0.150252,6.671062
std,24.318410,0.171268,23.805701
min,0.000000,0.000000,0.000000
25%,3.106016,0.046945,1.662911
50%,5.714600,0.095424,3.631697
75%,10.156388,0.188547,7.259309
max,1203.421328,2.667248,1248.145662


In [ ]:
train_df[['mean_joint_velocity', 'mean_angular_velocity', 'com_velocity']].head(10)

,mean_joint_velocity,mean_angular_velocity,com_velocity
0,0.000000,0.000000,0.000000
1,2.094620,0.061670,2.002386
2,1.985577,0.032278,1.257726
3,2.086352,0.025542,0.676110
4,2.901474,0.068981,1.580504
5,2.897861,0.065992,1.208857
6,3.148797,0.175249,1.366290
7,5.215515,0.074504,0.841870
8,3.096196,0.062893,1.431392
9,14.257150,0.787984,5.764767


In [ ]:
train_df.to_csv('output_train_added.csv', index=False)

In [ ]:
valid_df['mean_joint_velocity'] = compute_mean_joint_velocity(valid_df)
valid_df['mean_angular_velocity'] = compute_mean_angular_velocity(valid_df)
valid_df['com_velocity'] = compute_com_velocity(valid_df)

valid_df[['mean_joint_velocity', 'mean_angular_velocity', 'com_velocity']].describe()

,mean_joint_velocity,mean_angular_velocity,com_velocity
count,59200.000000,59200.000000,59200.000000
mean,9.941170,0.161870,7.734150
std,33.283393,0.184750,33.520994
min,0.000000,0.000000,0.000000
25%,3.174430,0.050809,1.713164
50%,5.803062,0.101708,3.725228
75%,10.316206,0.203240,7.518337
max,1204.752760,2.336046,1248.064123


In [ ]:
valid_df[['mean_joint_velocity', 'mean_angular_velocity', 'com_velocity']].head(10)

,mean_joint_velocity,mean_angular_velocity,com_velocity
0,0.000000,0.000000,0.000000
1,1.657754,0.029526,1.802235
2,3.047907,0.071306,1.078983
3,1.658001,0.030195,0.179968
4,1.822419,0.021735,1.480708
5,1.683423,0.030302,0.921544
6,1.865386,0.029387,1.343410
7,3.851858,0.057987,3.577288
8,3.306175,0.029899,2.511116
9,2.559601,0.069672,1.306130


In [ ]:
valid_df.to_csv('output_val_added.csv', index=False)